In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [40]:
import pinout
from lsm6dso import LSM6DSO
from machine import I2C, Pin, Timer
import time
from common import task

In [55]:
i2c = I2C(0, scl=Pin(pinout.scl), sda=Pin(pinout.sda), freq=1000000)
mems = LSM6DSO(i2c)
mems.scale_a('16g')

acc_filename = '/user/acc.csv'
freq = 30


def build_cb(buf):
    def measure_store(tim):
        t = time.ticks_us()
        val = mems.get_a()
    
        buf.append({
            'dt': t,
            'acc': tuple(val)
        })
    return measure_store
        
@task('1: Acc collecting')
def task_1_acc_collecting(duration):
    with open(acc_filename, 'w') as f:
        f.write("#dt, x, y, z\n")
    
    buf = list()
    
    tim = Timer(0)
    tim.init(period=int(1000/freq), mode=Timer.PERIODIC, callback=build_cb(buf))
    
    time.sleep(duration)
    tim.deinit()
    
    
    with open(acc_filename, 'a') as f:
        for m in buf:
            f.write(f"{m['dt']}, {m['acc'][0]}, {m['acc'][1]}, {m['acc'][2]}\n")
    
    with open(acc_filename, 'r') as f:
        print(f.read())

    
task_1_acc_collecting(1)

**************************************************
Start Of Task #1: Acc collecting
**************************************************
#dt, x, y, z
997901446, -82, -62, 1002
997934386, -167, -122, 2005
997967378, -167, -122, 2005
998000374, -81, -62, 1001
998033378, -81, -62, 1001
998066378, -81, -62, 1001
998099378, -82, -62, 1003
998132374, -82, -62, 1003
998165378, -39, -28, 1027
998198374, -39, -28, 1027
998231378, -39, -28, 989
998264374, -319, 28, 989
998297378, -319, 28, 989
998330374, -478, -415, 1600
998363378, -478, -415, 1600
998396374, -454, -156, 1002
998429378, -454, -156, 1002
998462374, -454, -156, 1002
998495378, -524, -260, 777
998528374, -524, -260, 777
998561378, -672, -421, 556
998594374, -672, -421, 556
998627378, -672, -421, 556
998660374, -732, -461, 399
998693378, -732, -461, 399
998726374, -721, -384, -111
998759374, -721, -384, -111
998792370, -721, -384, -376
998825374, -644, -360, -376
998858370, -644, -360, -376

*******************************************